In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr 20 11:44:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
!gdown --id 13wBO3baamOfaJnFBK6xyFxsC7AD0eT-F
!gdown --id 13sU9VWS_WISO_e8wU8S7zTo04qNpixrc
!gdown --id 13sg-tQrqtMJ1vmU3DhmaYmtg792eIS1Q

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13wBO3baamOfaJnFBK6xyFxsC7AD0eT-F
To: /content/prachatai-train.csv
100% 7.86M/7.86M [00:00<00:00, 110MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=13sU9VWS_WISO_e8wU8S7zTo04qNpixrc
To: /content/prachatai-dev.csv
100% 990k/990k [00:00<00:00, 97.9MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: h

In [4]:
!pip install sentencepiece

In [5]:
import pandas as pd
train_df = pd.read_csv('prachatai-train.csv')
dev_df = pd.read_csv('prachatai-dev.csv')
test_df = pd.read_csv('prachatai-test.csv')

train_df['title tokens'] = train_df['title tokens'].apply(lambda x: x.replace('|', ''))
dev_df['title tokens'] = dev_df['title tokens'].apply(lambda x: x.replace('|', ''))
test_df['title tokens'] = test_df['title tokens'].apply(lambda x: x.replace('|', ''))


labels = list(set(train_df['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
train_df['label'] = train_df['label'].apply(lambda x: label_to_idx[x])
dev_df['label'] = dev_df['label'].apply(lambda x: label_to_idx[x])
test_df['label'] = test_df['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label

In [6]:
label_list

['การเมือง', 'สังคม', 'เศรษฐกิจ', 'ต่างประเทศ', 'คุณภาพชีวิต', 'สิ่งแวดล้อม']

In [7]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,

    Trainer,
    TrainingArguments,
)

from datasets import (
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)

In [8]:
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
model_name = 'clicknext/phayathaibert'
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           config=config)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at clicknext/phayathaibert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
features= Features({
                    #"title tokens": Sequence(Value(dtype='string')),
                    "title tokens": Value(dtype='string'),

                    "label": ClassLabel(names=label_list)
                })
d = DatasetDict({'train': Dataset.from_pandas(train_df, features=features),
                 'dev': Dataset.from_pandas(dev_df, features=features),
                 'test': Dataset.from_pandas(test_df, features=features)})

In [10]:
d

DatasetDict({
    train: Dataset({
        features: ['title tokens', 'label'],
        num_rows: 37890
    })
    dev: Dataset({
        features: ['title tokens', 'label'],
        num_rows: 4736
    })
    test: Dataset({
        features: ['title tokens', 'label'],
        num_rows: 4737
    })
})

In [11]:
d['train'][0]

{'title tokens': 'กมธ.แจงมติเซ็ตซีโร่ กกต.หวั่นเป็นปลา 2 น้ำทำงานลำบาก ย้ำไม่มีใบสั่งจาก คสช.',
 'label': 0}

In [12]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples["title tokens"],
                                 is_split_into_words=False,
                                 truncation=True,
                                 max_length=50)
    return tokenized_inputs
tokenized_datasets = d.map(tokenize, batched=True)


Map:   0%|          | 0/37890 [00:00<?, ? examples/s]

Map:   0%|          | 0/4736 [00:00<?, ? examples/s]

Map:   0%|          | 0/4737 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 37890
    })
    dev: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4736
    })
    test: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4737
    })
})

In [16]:
tokenized_datasets['train'][0]['input_ids']

[5,
 10,
 4517,
 11,
 4189,
 3907,
 6141,
 21034,
 13517,
 11,
 4237,
 17,
 769,
 25068,
 10,
 143,
 258,
 2036,
 10,
 2417,
 159,
 12987,
 32,
 10,
 871,
 11,
 6]

In [13]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids'])

['<s>',
 '▁',
 'กมธ',
 '.',
 'แจง',
 'มติ',
 'เซ็ต',
 'ซีโร่',
 '▁กกต',
 '.',
 'หวั่น',
 'เป็น',
 'ปลา',
 '▁2',
 '▁',
 'น้ํา',
 'ทํางาน',
 'ลําบาก',
 '▁',
 'ย้ํา',
 'ไม่มี',
 'ใบสั่ง',
 'จาก',
 '▁',
 'คสช',
 '.',
 '</s>']

In [14]:
model_name.split('/')[-1]

'phayathaibert'

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 37890
    })
    dev: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4736
    })
    test: Dataset({
        features: ['title tokens', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4737
    })
})

In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)
#tokenized_datasets = tokenized_datasets.remove_columns(["title tokens"])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['title tokens', 'label', 'input_ids', 'attention_mask']

In [42]:
tokenized_datasets['dev'][20]

{'title tokens': 'แจ้งความ ปธ.ที่ปรึกษาวัดกัลยาณ์ ม.112',
 'label': 4,
 'input_ids': [5,
  10,
  2340,
  10,
  86,
  417,
  11,
  2498,
  317,
  15681,
  9126,
  10,
  76,
  11,
  4954,
  6],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [43]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k != 'title tokens'}
[len(x) for x in samples["input_ids"]]

[28, 16, 26, 8, 12, 34, 23, 30]

In [44]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 40]),
 'attention_mask': torch.Size([8, 40]),
 'labels': torch.Size([8])}

In [17]:
!pip install datasets evaluate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00


In [46]:
import evaluate

In [18]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [48]:
args = TrainingArguments(
    output_dir='prachatai-headline'
)

args2 = TrainingArguments(
    output_dir='prachatai-headline-2',
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    eval_strategy='steps',
    eval_steps=100,
    logging_steps=50,
    save_strategy='steps',
    report_to="wandb"
)

#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np
def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: attapol (nlp-chula) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
100,0.970500,0.939039,0.699747
200,0.923300,0.849031,0.713894
300,0.860700,0.813832,0.726351
400,0.799100,0.759910,0.736275
500,0.789400,0.728619,0.744088
600,0.739200,0.716414,0.749155
700,0.737100,0.694458,0.752534
800,0.723300,0.690309,0.756123
900,0.699400,0.676626,0.759291
1000,0.705600,0.669618,0.764147


TrainOutput(global_step=1186, training_loss=0.7915203551216833, metrics={'train_runtime': 690.4052, 'train_samples_per_second': 109.762, 'train_steps_per_second': 1.718, 'total_flos': 1481425999814592.0, 'train_loss': 0.7915203551216833, 'epoch': 2.0})

In [57]:

predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)


In [58]:
predictions.label_ids

array([4, 2, 4, ..., 4, 2, 4])

In [59]:
preds

array([4, 4, 4, ..., 4, 4, 0])

In [60]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

In [61]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

              precision    recall  f1-score   support

    การเมือง       0.79      0.91      0.85      3289
 คุณภาพชีวิต       0.56      0.33      0.42       290
  ต่างประเทศ       0.62      0.72      0.67       475
       สังคม       0.00      0.00      0.00       139
 สิ่งแวดล้อม       0.52      0.28      0.37       427
    เศรษฐกิจ       0.44      0.07      0.12       117

    accuracy                           0.75      4737
   macro avg       0.49      0.39      0.40      4737
weighted avg       0.71      0.75      0.72      4737



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [62]:
input = tokenizer("นักวิชาการเสนอใช้ 'สวัสดิการชราภาพถ้วนหน้าแบบยืดหยุ่น' ค้านพิสูจน์ความจนก่อนรับสิทธิ", return_tensors="pt")

In [63]:
input.to('cuda')
outputs = trainer.model(**input)
logits = outputs.logits
predicted_class = logits.argmax().item()

In [64]:
labels[predicted_class]

'คุณภาพชีวิต'